In [2]:
import tabula
import pandas as pd
import numpy as np
import re
import random
#import PyPDF2
from IPython.display import HTML
import os

This file contains a list of Keywords that signal a field that we are interested in. - Random Sample listed here.

In [3]:
with open('./data/keyword_fields.txt') as f:
    data = f.readlines()
    data.sort(key=len, reverse=True)
field_regex = re.compile('|'.join([dt.replace('\n', '') for dt in data if dt != '\\w+ #\n']).replace(' ', '\s'))

Very Basic Regex to Extract flight dates.

In [4]:
date_range_regex = re.compile(r'[0-9]{2}\/[0-9]{2}\/[0-9]{2}\s*-\s*[0-9]{2}\/[0-9]{2}\/[0-9]{2}')
date_regex = re.compile(r'[0-9]{2}\/[0-9]{2}\/[0-9]{2}')

In [6]:
def showPDF(filepath):
    return HTML('<iframe src="%s" width=1000 height=400></iframe>' % (filepath))

current_file = './data/37f0827f-6409-41f7-b418-3580e3bd1b50.pdf'
showPDF(current_file)

Convert the above file into a text based CSV using Tabula - this currently calls the JVM directly, in production using something like Drip natively might reduce load times by keeping the JVM alive. 

In [74]:
tabula.convert_into(current_file, output_path='./test.txt', output_format='csv', guess=False, pages='all', nospreadsheet=True)

Read the CSV into a Pandas Dataframe - not entirely sure that this is necesseary, but helps to convert into matrix later.

Also, look at the top 30% of the dataframe for the values we want. Sometimes the Total value is only available in the last page - so might look at the top 30% and the bottom 30% of the dataframe.

In [7]:
df = pd.read_csv('./test.txt', names=range(0,7))

Two Extraction Methods:

**Lateral**: Look for adjacent field. - Needed for some types.

**Medial**: Look for above and below. - Default.

In [71]:
def readPdfToText(input1):
    os.system("pdftotext -layout '%s' '%s'" % (input1, 'test.txt'))
    with open('./test.txt', 'r') as fp:
        data = fp.readlines()
readPdfToText('./data/fee68b92-3b27-4b95-87ed-1a6106f3ad95.pdf')

In [35]:
df_rows = []
for line in data:
    field_lines = [field for field in re.split('\s{2,}', line.replace('\n', '')) if field != '']
    if len(field_lines) > 1:
        df_rows.append(field_lines)
df = pd.DataFrame.from_records(df_rows)

In [76]:
a = df.as_matrix()
fields_meta = df.applymap(lambda x: field_regex.findall(x) if type(x) == str else False).as_matrix()

In [78]:
def extractFields(fields_meta, extraction_method):
    extracted_fields = []
    (lx,ly) = fields_meta.shape
    for x in range(0, lx):
        for y in range(0,ly):
            current_field = fields_meta[x,y]
            if current_field:
                if extraction_method == 'medial' and x < lx-1:
                    target_field = a[x+1,y]
                elif extraction_method == 'lateral' and y < ly-1:
                    target_field = a[x,y+1]
                extracted_fields.append((current_field,target_field))
    return extracted_fields
extracted_fields = extractFields(fields_meta, 'medial')

In [80]:
extracted_fields

[(['Contract / Revision', 'Alt Order #'], '1559701  / 08329440'),
 (['Product'], '(419) 248-1111 PORTMAN FOR SENATE'),
 (['Contract Dates'], '08/16/16 - 08/22/16'),
 (['Estimate #'], '3386'),
 (['Advertiser'], 'POL/Portman, R/United States Senate'),
 (['Original Date / Revision'], '07/21/16  / 07/21/16'),
 (['Billing Cycle', 'Billing Calendar', 'Cash/Trade'],
  'EOM/EOC Broadcast Cash'),
 (['Property Account Executive', 'Sales Office'],
  'WTOL Dorie Penafiel HRP-Washingto'),
 (['Special Handling'], 'Cash In Advance'),
 (['Demographic'], 'Adults 35+'),
 (['Advertiser Code', 'Product'], '1021 280 316'),
 (['Agency Ref', 'Advertiser Ref'], nan),
 (['Spots'], '*Line Ch Start Date End Date Description Time Days Length Week'),
 (['Start Date', 'End Date'],
  'N 1 WTOL 08/19/16 08/19/16 NEWS 11 @ NOON 1158am-1230pm :30'),
 (['Rate', 'Spots'], 'NM 1'),
 (['Start Date', 'End Date', 'Spots', 'Rate'],
  'Week: 08/15/16 08/21/16 ----1-- 1 $500.00'),
 (['Start Date', 'End Date', 'Spots', 'Rate'],


In [82]:
def cleanFields(extracted_fields, lazy=False):
    finalized_meta = {}
    for field, value in extracted_fields:
        if not pd.isnull(value):
            if field[0] == 'Contract / Revision' and not pd.isnull(value):
                finalized_meta['altOrder'] = value.split(' ')[0] if type(value) == str else value
            if field[0] in ['Contract Dates', 'Schedule Dates']:
                finalized_meta['flightDates'] = value
            if field[0] == 'Demographic':
                finalized_meta['Demographic'] = value
            if len(field) == 1 and field[0] == 'Advertiser':
                finalized_meta['Advertiser'] = value
            if len(field) == 1:
                if ' / ' in field[0]:
                    finalized_meta.update(dict(zip(field[0].split(' / '), value.split(' / '))))
            if len(field) > 2:
                if 'Period' in field and 'Spots' in field:
                    matched_date = date_range_regex.findall(value)[0]
                    value = value.replace(matched_date, matched_date.replace(' ', ''))
                    finalized_meta.update(dict(zip(field, value.split(' '))))
                    #if field[2] == 'Gross Amount':
                    #    finalized_meta['total'] = value.split(' ')[-1]
                #finalized_meta[field]
            if lazy:
                if len(field) == 1 and not pd.isnull(value):
                    finalized_meta[field[0].strip()] = value.strip()
    print(finalized_meta)
cleanFields(extracted_fields, lazy=True)

{'Advertiser': 'POL/Portman, R/United States Senate', 'Totals': 'Signature: Date:', 'Original Date / Revision': '07/21/16  / 07/21/16', 'Station': 'otherwise, on contracts, insertion orders, copy instructions or any correspondence when such conflict with the above terms and conditions.', 'Estimate #': '3386', 'Contract Dates': '08/16/16 - 08/22/16', 'Net Amount': '$9,350.00', 'Demographic': 'Adults 35+', 'Agency': '($1,650.00)', 'Product': '(419) 248-1111 PORTMAN FOR SENATE', 'Special Handling': 'Cash In Advance', 'Gross Amount': '$11,000.00', 'flightDates': '08/16/16 - 08/22/16', 'Original Date': '07/21/16 ', 'Period': '08/01/16-08/22/16', 'Spots': '6', 'altOrder': '1559701', 'Revision': '07/21/16'}


Desired Output:

{
"total": "7480",
"flightDates":"08/19/16-08/29/16",
"order": "920710",
"altOrder": "25269827"
}

In [192]:
print('Final Output', finalized_meta)

Final Output {'Original Date': '06/16/16  /', 'Period': '08/29/16-09/25/16', 'Gross Amount': '$22,070.00', 'Agency': '($3,310.50)', 'Revision': '08/15/16', 'Spots': '43', 'Demographic': 'Adults 25-54', 'altOrder': '156304', 'flightDates': '09/20/16 - 09/26/16', 'Advertiser': 'Portman/Republican/Sena'}
